In [1]:
from IPython.display import clear_output
from google.colab import drive
drive.mount('/content/drive')
clear_output()

In [2]:
from keras.models import Sequential
from keras.layers.pooling.global_max_pooling2d import GlobalMaxPooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Dense
from keras import backend as K
from keras import regularizers
import tensorflow as tf

import numpy as np
import pickle
import random

In [3]:
with open('/content/drive/MyDrive/furniture_classification/dataset_resized.pkl', 'rb') as file:
    dataset = pickle.load(file)
    labels = dataset[1]
    images = dataset[0]

In [5]:
train_labels = []
train_images = []
val_labels = []
val_images = []
for i in range(len(images)):
  train_labels.append(labels[i][:7])
  train_images.append(images[i][:7])
  val_labels.append(labels[i][7:])
  val_images.append(images[i][7:])

In [6]:
def batch_gen():
  zipped = list(zip(train_images, train_labels))
  while True:
    random.shuffle(zipped)
    images = [i[0] for i in zipped]
    labels = [i[1] for i in zipped]
    for i in range(len(images)):
      yield images[i][:5], labels[i][:5]
      yield images[i][5:], labels[i][5:]
    # yield train_images[i], train_labels[i]

In [7]:
def val_gen():
  zipped = list(zip(val_images, val_labels))
  while True:
    random.shuffle(zipped)
    images = [i[0] for i in zipped]
    labels = [i[1] for i in zipped]
    for i in range(len(images)):
      yield images[i][:], labels[i][:]

In [44]:
model = Sequential()
model.add(Conv2D(32, 2, input_shape=(None, None, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, 2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(64, 3))
model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(256))
# model.add(Activation('relu'))
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.1))
model.add(Dense(3,kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))
model.add(Activation('softmax'))

In [45]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, None, None, 32)    160       
                                                                 
 activation_35 (Activation)  (None, None, None, 32)    0         
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, None, None, 32)   0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, None, None, 32)    4128      
                                                                 
 activation_36 (Activation)  (None, None, None, 32)    0         
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, None, None, 32)   0         
 g2D)                                                 

In [46]:
model.compile(loss='sparse_categorical_crossentropy',
              # optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              optimizer='adam',
              metrics=['accuracy'])
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.hdf5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [47]:
history = model.fit(batch_gen(), epochs=50, steps_per_epoch=200, validation_steps=30, validation_data =val_gen(), callbacks=[model_checkpoint_callback])

Epoch 1/50
200/200 [==============================] - 6s 21ms/step - loss: 3.7678 - accuracy: 0.4029 - val_loss: 0.9938 - val_accuracy: 0.4556
Epoch 2/50
200/200 [==============================] - 4s 21ms/step - loss: 0.9405 - accuracy: 0.5443 - val_loss: 0.9802 - val_accuracy: 0.4778
Epoch 3/50
200/200 [==============================] - 4s 21ms/step - loss: 0.8429 - accuracy: 0.6371 - val_loss: 0.7688 - val_accuracy: 0.7556
Epoch 4/50
200/200 [==============================] - 4s 20ms/step - loss: 0.5580 - accuracy: 0.7814 - val_loss: 0.8830 - val_accuracy: 0.6444
Epoch 5/50
200/200 [==============================] - 4s 20ms/step - loss: 0.4959 - accuracy: 0.8071 - val_loss: 0.8781 - val_accuracy: 0.6778
Epoch 6/50
200/200 [==============================] - 4s 21ms/step - loss: 0.3733 - accuracy: 0.8629 - val_loss: 0.7756 - val_accuracy: 0.7778
Epoch 7/50
200/200 [==============================] - 4s 20ms/step - loss: 0.2336 - accuracy: 0.9086 - val_loss: 0.6100 - val_accuracy: 0.7111

In [29]:
model.load_weights('best_model.hdf5')

In [48]:
zipped_val = zip(val_images, val_labels)
model.evaluate(zipped_val)

30/30 [==============================] - 0s 8ms/step - loss: 0.8791 - accuracy: 0.7889


[0.8791431188583374, 0.7888888716697693]